In [ ]:
import gym
import numpy as np
import random
import tensorflow as tf
from collections import deque
from skimage.transform import resize

In [ ]:
env = gym.make('BreakoutDeterministic-v4')

input_size = 84*84*4
output_size = env.action_space.n

HEIGHT = 84
WIDTH = 84
HISTORY_SIZE = 4

dis = 0.99
REPLAY_MEMORY = 400000

In [ ]:
class DQN :
    def __init__(self, session, input_size, output_size, name="main") :
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.height = 84
        self.width = 84
        self.history_size = 4
        self.net_name = name
        self._build_network()
    
    def pre_proc(X):
        x = np.uint8(resize(rgb2gray(X), (84, 84), mode='reflect') * 255)
        return x
    
    def _build_network(self, l_rate=0.00025) :
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder('float', [None, self.height, self.width, self.history_size])
            self.a = tf.placeholder('int64', [None])
            
            f1 = tf.get_variable("f1", shape=[8, 8, 4, 16],
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())
            f2 = tf.get_variable("f2", shape=[4, 4, 16, 32],
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())
            w1 = tf.get_variable("w1", shape=[9*9*32, 256],
                                 initializer=tf.contrib.layers.xavier_initializer())
            w2 = tf.get_variable("w2", shape=[256, self.output_size],
                                 initializer=tf.contrib.layers.xavier_initializer())
            
            c1 = tf.nn.relu(tf.nn.conv2d(self._X, f1, strides=[1, 4, 4, 1], padding="VALID"))
            c2 = tf.nn.relu(tf.nn.conv2d(c1, f2, strides=[1, 2, 2, 1], padding="VALID"))
            
            l1 = tf.reshape(c2, [-1, w1.get_shape().as_list()[0]])
            l2 = tf.nn.relu(tf.matmul(l1, w1))
            
            self._Qpred = tf.matmul(l2, w2)
        
        action_one_hot = tf.one_hot(self.a, self.output_size, 1.0, 0.0)
        q_val = tf.reduce_sum(tf.multiply(self._Qpred, action_one_hot), axis=1)
        
        self._Y = tf.placeholder(shape=[None, self.output_size], dtype=tf.float32)
    
        self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
    
        self._train = tf.train.AdamOptimizer(learning_rate = l_rate).minimize(self._loss)
    
    def predict(self, state):
        return self.session.run(self._Qpred, feed_dict={self._X : np.reshape(state, [-1, 84, 84, 4])})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train],
                                feed_dict={self._X : np.reshape(x_stack, [-1, 84, 84, 4]), self._Y : y_stack})    

In [ ]:
def replay_train (mainDQN, targetDQN, train_batch) :
    x_stack = np.empty(0).reshape(0, 84, 84, 4)
    y_stack = np.empty(0).reshape(0, output_size)
    
    for history, action, reward, done in train_batch:
        Q = mainDQN.predict(history[:, :, :4])
        
        if done :
            Q[0,action] = reward
        else :
            action0 = np.argmax(mainDQN.predict(history[:, :, 1:]))
            Q[0,action] = reward + dis * (targetDQN.predict(history[:, :, 1:])[0, action0])
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, np.reshape(history[:, :, :4], [-1, 84, 84, 4])])
    
    return mainDQN.update(x_stack, y_stack)

In [ ]:
def get_copy_var_ops(dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
    
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
    
    return op_holder

In [ ]:
def bot_play(mainDQN) :
    s = env.reset()
    reward_sum = 0
    while True :
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s,reward,done,_ = env.step(a)
        reward_sum += reward
        
        if done :
            print ("Total score : {}".format(reward_sum))
            break

In [ ]:
def rgb2gray(X) :
    return np.mean(X, axis=2).astype(np.uint8)

def conversion_image(X) :
    x = np.uint8(resize(rgb2gray(X), (HEIGHT, WIDTH), mode='reflect') * 255)
    return x

def init_history_conv(history, state) :
    for i in range(5):
        history[:, :, i] = conversion_image(state)

In [ ]:
def main():
    
    max_episodes = 1000
    
    replay_buffer = deque()
    
    
    with tf.Session() as sess :
        mainDQN = DQN(sess, input_size, output_size, name="main")
        targetDQN = DQN(sess, input_size, output_size, name="target")
        
        tf.initialize_all_variables().run()
        
        
        copy_ops = get_copy_var_ops(dest_scope_name = "target",
                                   src_scope_name = "main")
        sess.run(copy_ops)
    
        
        step_count_total = 0
        for episode in range(max_episodes):
            e = 1. / ((episode / 10) + 1)
            done = False
            step_count = 0
            state = env.reset()
            
            history = np.zeros([84, 84, 5], dtype=np.uint8)
            init_history_conv(history, state)
            
            while not done:
                env.render()
                
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(mainDQN.predict(np.reshape(history[:, :, :4], [-1, 84, 84, 4])))
    
                next_state, reward, done, _ = env.step(action)
                history[:, :, 4] = conversion_image(next_state)
                
                if done:
                    reward = -1
                
                replay_buffer.append((np.copy(history[:,:,:]), action, reward, done))
                history[:, :, :4] = history[:, :, 1:]
                if len(replay_buffer) > REPLAY_MEMORY:
                    replay_buffer.popleft()
    
                if reward > 0 :
                    step_count += reward
                if step_count > 100 :
                    break
                
            print("episode: {}   steps: {}".format(episode, step_count))
            step_count_total += step_count
    
            if episode % 5 == 0 and episode > 0:
                if step_count_total == 500:
                    break
                
                for _ in range(10):
                    minibatch = random.sample(replay_buffer, 8)
                    loss, _ = replay_train(mainDQN, targetDQN, minibatch)
    
                print ("Loss :  ", loss)
                print ("Step Total 5 :  ", step_count_total)
                step_count_total = 0
                sess.run(copy_ops)
        
        print ("Simulation--------")
        bot_play(mainDQN)

In [ ]:
main()